### Install packages

In [1]:
install.packages(c("e1071", "kernlab", "randomForest"))
library("e1071")
library("kernlab")
library("caret")
library("ellipse")

Warning message in install.packages(c("e1071", "kernlab", "randomForest")):
“installation of package ‘kernlab’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error in library("kernlab"): there is no package called ‘kernlab’


### Load iris dataset

In [ ]:
# attach the iris dataset to the environment
data(iris)
# rename the dataset
dataset <- iris

### Create a validation datastet

In [ ]:
# create a list of 80% of the rows in the original dataset we can use for training
validation_index <- createDataPartition(dataset$Species, p=0.80, list=FALSE)
# select 20% of the data for validation
validation <- dataset[-validation_index,]
# use the remaining 80% of data to training and testing the models
dataset <- dataset[validation_index,]

### Look at characteristics of data

In [ ]:
# dimensions of dataset
dim(dataset)

In [ ]:
# list types for each attribute
sapply(dataset, class)

In [ ]:
# take a peek at the first 5 rows of the data
head(dataset)

In [ ]:
# list the levels for the class
levels(dataset$Species)

In [ ]:
# summarize the class distribution
percentage <- prop.table(table(dataset$Species)) * 100
cbind(freq=table(dataset$Species), percentage=percentage)

In [ ]:
# summarize attribute distributions
summary(dataset)

### Visualize Dataset

#### Univariate Plots

In [ ]:
# split input and output
x <- dataset[,1:4]
y <- dataset[,5]

In [ ]:
# boxplot for each attribute on one image
par(mfrow=c(1,4))
  for(i in 1:4) {
  boxplot(x[,i], main=names(iris)[i])
}

In [ ]:
# barplot for class breakdown
plot(y)

#### Multivariate Plots

In [ ]:
# scatterplot matrix
featurePlot(x=x, y=y, plot="ellipse")

In [ ]:
# box and whisker plots for each attribute
featurePlot(x=x, y=y, plot="box")

In [ ]:
# density plots for each attribute by class value
scales <- list(x=list(relation="free"), y=list(relation="free"))
featurePlot(x=x, y=y, plot="density", scales=scales)

### Evaluating Algorithms

#### Test harness

In [ ]:
# Run algorithms using 10-fold cross validation
control <- trainControl(method="cv", number=10)
metric <- "Accuracy"

#### Build models and evaluate each

In [ ]:
# a) linear algorithms
set.seed(7)
fit.lda <- train(Species~., data=dataset, method="lda", metric=metric, trControl=control)
# b) nonlinear algorithms
# CART
set.seed(7)
fit.cart <- train(Species~., data=dataset, method="rpart", metric=metric, trControl=control)
# kNN
set.seed(7)
fit.knn <- train(Species~., data=dataset, method="knn", metric=metric, trControl=control)
# c) advanced algorithms
# SVM
set.seed(7)
fit.svm <- train(Species~., data=dataset, method="svmRadial", metric=metric, trControl=control)
# Random Forest
set.seed(7)
fit.rf <- train(Species~., data=dataset, method="rf", metric=metric, trControl=control)